## Adult Dataset Part 1: Cleaning and Preparation

#### Objectives:
1. Preprocess the data and store the cleaned dataset as adult_clean.csv
2. Load clean dataset and test supervised/unsupervised models
    - Goal: Determine best model to predict if new entry earns >50k or <50k with supervised learning
    - Goal: Find insights and patterns in data using unspuervised learning
3. Apply k-folds cross validation
    - Goal: Evaluate robustness of results across different models.
    - Goal: Use and justify different error metrics


#### Summary of dataset

Datasource: https://archive.ics.uci.edu/ml/datasets/adult

Feature descrptions copied below from the UCI website:

- age: continuous. 
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
- fnlwgt: Continuous. A weighting assigned from the sampling process.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
- education-num: continuous. 
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
- sex: Female, Male. 
- capital-gain: continuous.
- capital-loss: continuous. 
- hours-per-week: continuous. 
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- target: >50K, <=50K.

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [41]:
#original data does not come with headers. add headers based on the documentation descrption
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
           'hours-per-week', 'native-country', 'target']

In [42]:
data = pd.read_csv('./data/adult.data', names = headers)
df = data.copy()

In [43]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  target          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


On first look, it appears there are no missing values. **However the dataset descrption indicates that missing values have been flled with '?'.** Figure out what to do here :)

In [112]:
headers = [df.workclass] 
print(len(df.workclass))
#big_list = []
test = []

special_characters = "?"

for i in headers:
    for j in i:
        #print(type(j))
        if special_characters not in j:
            test.append(j)
            
test
#print(len(test))
#print(test[11527])
#print(df.head(11527))

32561


[' State-gov',
 ' Self-emp-not-inc',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Self-emp-not-inc',
 ' Private',
 ' Private',
 ' Private',
 ' State-gov',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Self-emp-not-inc',
 ' Private',
 ' Private',
 ' Self-emp-not-inc',
 ' Private',
 ' Private',
 ' Federal-gov',
 ' Private',
 ' Private',
 ' Local-gov',
 ' Private',
 ' Private',
 ' Private',
 ' Local-gov',
 ' Private',
 ' Private',
 ' Federal-gov',
 ' State-gov',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Self-emp-not-inc',
 ' Private',
 ' Self-emp-not-inc',
 ' Private',
 ' Private',
 ' Private',
 ' Federal-gov',
 ' Private',
 ' Private',
 ' State-gov',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Federal-gov',
 ' Self-emp-inc',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Private',
 ' Self-emp-inc',
 ' Private',
 ' Private',
 ' Self-emp

In [91]:
#code
#df[df.eq('Private').any(1)]
#print(df.head(11527))
#value = re.sub(r"?", "nan", df)
print(df.columns)

def data_checker(col_list):
    big_list = []
    for i in col_list:
        for j in i:
            i = []
            if j != 0:
                a = i.append(j)
                big_list.append(a)
    return big_list
        
#headers = [df.age, df.workclass]         
            
#data_checker(headers)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'target'],
      dtype='object')


Before deciding **how to deal with the missing values want to know how they influence the data**. For example, if workclass and occupation for example both have missing values (in the same tuple) it would be reasonable to drop the data.

In [ ]:
#code

**What % of the dataset do nans represent?** What is the **best approach** for dealing with the nan's?

In [ ]:
#plot the categroical columns to see what we are dealing with


In [ ]:
#plot the numerical columns


#### Additional Steps:
1. Apply statistical tests to determine if dropping values changes the structure of the distribution in the workplace, occupation, and native-country categories i.e. t-test for differnce of modes. 
2. Compare filling the workplace and occupation columns with mode values and applying same statistical tests.

In [ ]:
#train / test... you know the drill

# Discussion of results